<a href="https://colab.research.google.com/github/VaishnaviMudaliar/deep-learning/blob/main/Resnet_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing the libraries

import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [2]:
# loading cifar 10 dataset

def data_loader(data_dir , batch_size, valid_size = 0.1 , shuffle = True , random_seed = 42 , test = False):
  normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

  #define transforms
  transform = transforms.Compose([
      transforms.Resize((224,224)),
      transforms.ToTensor(),
      normalize
  ])

  if test :
    dataset = datasets.CIFAR10(root=data_dir, train=False, download=True, transform=transform)
    data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    return data_loader

  train_dataset = datasets.CIFAR10(root = data_dir , train = True , download = True , transform = transform)
  valid_dataset = datasets.CIFAR10(root = data_dir , train = True , download = True , transform = transform)
  num_train = len(train_dataset)
  indices = list(range(num_train))
  split = int(np.floor(valid_size * num_train))

  if shuffle:
    np.random.seed(random_seed)
    np.random.shuffle(indices)

  train_idx, valid_idx = indices[split:], indices[:split]
  train_sampler = SubsetRandomSampler(train_idx)
  valid_sampler = SubsetRandomSampler(valid_idx)

  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler)

  valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, sampler=valid_sampler)

  return train_loader , valid_loader


  # CIFAR10 dataset

train_loader , valid_loader = data_loader(data_dir = './data', batch_size = 64)
test_loader = data_loader(data_dir = './data', batch_size = 64 , test = True)

100%|██████████| 170498071/170498071 [00:03<00:00, 50913793.95it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Files already downloaded and verified


In [3]:
# Residual Block

class ResidualBlock(nn.Module):
  def __init__(self, in_channels, out_channels, stride=1 , downsample = None):
    super(ResidualBlock, self).__init__()
    self.conv1 = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True)

    )
    self.conv2 = nn.Sequential(
        nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
        nn.BatchNorm2d(out_channels)
    )
    self.downsample = downsample
    self.relu = nn.ReLU()
    self.out_channels = out_channels

  def forward(self, x):
    residual = x
    out = self.conv1(x)
    out = self.conv2(out)
    if self.downsample:
      residual = self.downsample(x)
    out += residual
    out = self.relu(out)
    return out

In [4]:
# Building the ResNet

class ResNet(nn.Module):
  def __init__(self, block, layers, num_classes=10):
    super(ResNet, self).__init__()
    self.inplanes = 64
    self.conv1 = nn.Sequential(
        nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False),
        nn.BatchNorm2d(64),
        nn.ReLU(inplace=True)
    )
    self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
    self.layer0 = self._make_layer(block, 64, layers[0], stride=1)
    self.layer1 = self._make_layer(block, 128, layers[1], stride=2)
    self.layer2 = self._make_layer(block, 256, layers[2], stride=2)
    self.layer3 = self._make_layer(block, 512, layers[3], stride=2)
    self.avgpool = nn.AvgPool2d(7 , stride = 1)
    self.fc = nn.Linear(512 , num_classes)

  def _make_layer(self , block , planes , blocks , stride = 1):
    downsample = None
    if stride != 1 or self.inplanes != planes:
      downsample = nn.Sequential(
          nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=stride, bias=False),
          nn.BatchNorm2d(planes)
      )
    layers = [] # Initialize layers list outside the if block
    layers.append(block(self.inplanes, planes, stride, downsample))
    self.inplanes = planes
    for i in range(1, blocks):
      layers.append(block(self.inplanes, planes))
    return nn.Sequential(*layers)

  def forward(self, x): # De-indent the content of this method
    x = self.conv1(x)
    x = self.maxpool(x)
    x = self.layer0(x)
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)

    x = self.avgpool(x)
    x = x.view(x.size(0), -1)
    x = self.fc(x)
    return x

In [5]:

num_classes = 10
num_epochs = 20
batch_size = 16
learning_rate = 0.01

model = ResNet(ResidualBlock, [3, 4, 6, 3]).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.001, momentum = 0.9)

# Train the model
total_step = len(train_loader)


In [ ]:
# Training

import gc
total_step = len(train_loader)

for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    # Move tensors to configured device
    images = images.to(device)
    labels = labels.to(device)

    # Forward pass

    outputs = model(images)
    loss = criterion(outputs , labels)

    # backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    del images, labels, outputs
    gc.collect()

  print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
  torch.save(model.state_dict(), 'model.ckpt')

  # Validation
  with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in valid_loader:
      images = images.to(device)
      labels = labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
      del images, labels, outputs
    gc.collect()
    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

In [ ]:
# testing

with torch.no_grad():
  correct = 0
  total = 0
  for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    del images, labels, outputs
  gc.collect()
  print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))